In [0]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

Using TensorFlow backend.


In [0]:
text_file_name = "first_names.txt"

In [0]:
data = open(text_file_name,'r').read()
data = data.lower().strip()
chars = list(set(data))
vocab_size = len(chars)
print(len(data),vocab_size)

35147 29


In [0]:
m = sum(1 for line in open(text_file_name) if line.rstrip())
print(m)

4945


In [0]:
unique_chars = sorted(chars)
char_to_ix = {ch:i for i,ch in enumerate(unique_chars)}
ix_to_char = {i:ch for i,ch in enumerate(unique_chars)}
print(ix_to_char)

{0: '\n', 1: ' ', 2: '-', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'j', 13: 'k', 14: 'l', 15: 'm', 16: 'n', 17: 'o', 18: 'p', 19: 'q', 20: 'r', 21: 's', 22: 't', 23: 'u', 24: 'v', 25: 'w', 26: 'x', 27: 'y', 28: 'z'}


In [0]:
Tx = 12
x = np.zeros((m,Tx,vocab_size))
y = np.zeros((m,Tx,vocab_size))
with open(text_file_name) as f:
    examples = f.readlines()
example = [e.lower().strip() for e in examples]
np.random.shuffle(example)
for i in range(m):
    single_example = example[i]
    single_example_chars = [char_to_ix[c] for c in single_example]
    for j in range(Tx):
        if j<len(single_example_chars) and j!=0:
            x[i,j,single_example_chars[j]] = 1
            y[i,j-1,single_example_chars[j]] = 1  
x = np.asarray(x)
y = np.swapaxes(y,0,1)
y = y.tolist()
y = np.asarray(y)
print(x.shape)
print(y.shape)

(4945, 12, 29)
(12, 4945, 29)


In [0]:
n_a = 64
reshapor = Reshape((1, vocab_size))                        # Used in Step 2.B of djmodel(), below
LSTM_cell = LSTM(n_a, return_state = True)         # Used in Step 2.C
densor = Dense(vocab_size, activation='softmax')     # Used in Step 2.D

In [0]:
def djmodel(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras instance model with n_a activations
    """
    
    
    X = Input(shape=(Tx, n_values))
    
    
    
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    
    outputs = []
    
    
    for t in range(Tx):         
        x = Lambda(lambda x: x[:,t,:])(X)       
        x = reshapor(x)      
        a, _, c = LSTM_cell(inputs=x, initial_state=[a, c])        
        out = densor(a)        
        outputs.append(out)
        
    
    model = Model(inputs=[X,a0,c0],outputs=outputs)
    
    return model

In [0]:
model = djmodel(Tx , n_a, vocab_size)

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12, 29)       0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 29)           0           input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 29)        0           lambda_1[0][0]                   
                                                                 lambda_2[0][0]                   
                                                                 lambda_3[0][0]                   
                                                                 lambda_4[0][0]             

In [0]:
opt = Adam(learning_rate=0.01,beta_1=0.9,beta_2=0.999,decay=0.01)
model.compile(opt,"categorical_crossentropy",metrics=["accuracy"])

In [0]:
a0 = np.zeros((m,n_a))
c0 = np.zeros((m,n_a))

In [0]:
model.fit([x,a0,c0],list(y),epochs=1000)

Epoch 1/1000
4945/4945 [==============================] - 2s 394us/step - loss: 13.0550 - dense_1_loss: 0.0000e+00 - dense_1_accuracy: 0.2512 - dense_1_accuracy_1: 0.2726 - dense_1_accuracy_2: 0.1139 - dense_1_accuracy_3: 0.1347 - dense_1_accuracy_4: 0.1395 - dense_1_accuracy_5: 0.0797 - dense_1_accuracy_6: 0.0461 - dense_1_accuracy_7: 0.0202 - dense_1_accuracy_8: 0.0077 - dense_1_accuracy_9: 0.0018 - dense_1_accuracy_10: 6.0667e-04 - dense_1_accuracy_11: 0.0000e+00
Epoch 2/1000
4945/4945 [==============================] - 2s 391us/step - loss: 13.0549 - dense_1_loss: 0.0000e+00 - dense_1_accuracy: 0.2512 - dense_1_accuracy_1: 0.2726 - dense_1_accuracy_2: 0.1139 - dense_1_accuracy_3: 0.1347 - dense_1_accuracy_4: 0.1395 - dense_1_accuracy_5: 0.0797 - dense_1_accuracy_6: 0.0461 - dense_1_accuracy_7: 0.0202 - dense_1_accuracy_8: 0.0077 - dense_1_accuracy_9: 0.0018 - dense_1_accuracy_10: 6.0667e-04 - dense_1_accuracy_11: 0.0000e+00
Epoch 3/1000
4945/4945 [==============================] - 

In [0]:
def one_hot(x):
  k = K.argmax(x)
  k = tf.one_hot(indices=k,depth=27)
  k = RepeatVector(1)(k)
  return k

In [0]:
def name_inference_model(LSTM_cell, densor, vocab_size, n_a):
    
    x0 = Input((1, vocab_size))
    
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    outputs = []
    
        
    a, _, c = LSTM_cell(x,initial_state=[a,c])
        
    out = densor(a)
    #outputs.append(out)
    #x = Lambda(one_hot)(out)
        
    inference_model = Model(inputs=[x0,a0,c0],outputs=[out,a])
    
    return inference_model

In [0]:
inference_model = name_inference_model(LSTM_cell, densor, vocab_size, n_a)


In [0]:
inference_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1, 29)        0                                            
__________________________________________________________________________________________________
a0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
c0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 64), (None,  24064       input_2[0][0]                    
                                                                 a0[0][0]                   

In [0]:
x0 = np.zeros((1,1,vocab_size))
a0 = np.zeros((1,n_a))
c0 = np.zeros((1,n_a))

In [0]:
output,a = inference_model.predict([x0,a0,c0])
print(np.random.choice(list(range(vocab_size)),p=np.asarray(output).ravel()))

3


In [0]:
from keras.utils import to_categorical
def predict_sample_names(inference_model,x0,a0,c0):
  EOS = char_to_ix["\n"]
  indices =[]
  idx = -1
  i=0
  while(idx!= EOS and i!=12):
    pred,a = inference_model.predict([x0,a0,c0])
    idx = np.random.choice(list(range(vocab_size)),p=np.asarray(pred).ravel())
    indices.append(idx)
    x0 = np.zeros((1,1,vocab_size))
    x[1,1,idx]=1
    a0 = a
    i = i+1
  return indices



In [0]:
for i in range(5):
  sample_indices = predict_sample_names(inference_model,x0,a0,c0)
  text = ''.join(ix_to_char[x] for x in sample_indices)
  text = text[0].upper()+text[1:]
  print(text)

Ednlon-cdetb
Acrislrttcoi
Idatiaciyieu
Nhlelhipdlrs
A eziarpintn
